## Drug-Drug Interactions
DrugBank drug interacts with other DrugBank drug

In [2]:
import json
import csv
import pandas as pd
import parse_xml
from biomed_apis import *
from biomedkg_utils import *
from parse_xml import *
from lxml import etree
import xml.etree.ElementTree as ET

In [3]:
tree = ET.parse('input/full_drugbank.xml')
root = tree.getroot()

### DrugBank -interacts_with-> DrugBank 

In [4]:
drug2interacting_drugs = dict()
for i, ele in enumerate(root):
    # Main drug's DrugBank ID
    db_id = ParseXML.getID(ele)
    
    # Interacting drugs' DrugBank IDs
    interacting_drugs = ParseXML.getInteractingDrugs(ele)
    
    # Drug - Drug interaction
    drug2interacting_drugs[db_id] = interacting_drugs

In [78]:
output_edgefile_onerel_noweight('output/compound2compound/edges_drugbank-interacts_with->drugbank.csv',
                               ['Compound (DrugBank)', 'Compound (DrugBank)', 'Relationship'],
                               drug2interacting_drugs,
                                '-interacts_with->',
                               'DrugBank_Compound:',
                               'DrugBank_Compound:')

### MeSH -interacts_with-> MeSH 

In [74]:
db2mesh = json.load(open('output/compound2compound/db2mesh.json'))
meshdrug2interacting_drugs = dict()

for db, intdbs in drug2interacting_drugs.items():
    
    try:
        # Main MeSH
        mesh_mains = db2mesh[db]
        for mesh_main in mesh_mains:

            # Interacting MeSH
            for intdb in intdbs:
                mesh_ints = db2mesh[intdb]
                for mesh_int in mesh_ints:

                    # MeSH -interacts- MeSH Drug/Compound
                    meshdrug2interacting_drugs.setdefault(mesh_main, set()).add(mesh_int)         
    except:
        continue

In [77]:
output_edgefile_onerel_noweight('output/compound2compound/edges_MeSH-interacts_with->MeSH.csv',
                               ['Compound (MeSH)', 'Compound (MeSH)', 'Relationship'],
                               meshdrug2interacting_drugs,
                                '-interacts_with->',
                               'MeSH_Compound:',
                               'MeSH_Compound:')